In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import cv2
import torch
import random
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

In [ ]:
X_train = np.load('/content/drive/MyDrive/Colab Notebooks/CS671/Assignment_2/X_train.npy')
y_train = np.load('/content/drive/MyDrive/Colab Notebooks/CS671/Assignment_2/y_train.npy')
X_test = np.load('/content/drive/MyDrive/Colab Notebooks/CS671/Assignment_2/X_test.npy')
y_test = np.load('/content/drive/MyDrive/Colab Notebooks/CS671/Assignment_2/y_test.npy')
X_validation = np.load('/content/drive/MyDrive/Colab Notebooks/CS671/Assignment_2/X_validation.npy')
y_validation = np.load('/content/drive/MyDrive/Colab Notebooks/CS671/Assignment_2/y_validation.npy')

In [ ]:
plt.subplot(2,3,1),plt.imshow(X_train[0].reshape(28,28))
plt.subplot(2,3,2),plt.imshow(X_train[2277].reshape(28,28))
plt.subplot(2,3,3),plt.imshow(X_train[4554].reshape(28,28))
plt.subplot(2,3,4),plt.imshow(X_train[6831].reshape(28,28))
plt.subplot(2,3,5),plt.imshow(X_train[9108].reshape(28,28))

In [ ]:
"""
For selecting number of neurons in hidden layer
# https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw
"""
dummy = list(zip(X_train, y_train))
random.shuffle(dummy)
X_train, y_train = zip(*dummy)
X_train = np.array(X_train)
y_train = np.array(y_train)

dummy = list(zip(X_test, y_test))
random.shuffle(dummy)
X_test, y_test = zip(*dummy)
X_test = np.array(X_test)
y_test = np.array(y_test)

dummy = list(zip(X_validation, y_validation))
random.shuffle(dummy)
X_validation, y_validation = zip(*dummy)
X_validation = np.array(X_validation)
y_validation = np.array(y_validation)

In [ ]:
plt.subplot(2,3,1),plt.imshow(X_train[10].reshape(28,28))
print(y_train[10])

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='sigmoid', name='hidden_layer_1', input_shape=(784,)),
    tf.keras.layers.Dense(32, activation='sigmoid', name='hidden_layer_2'),
    tf.keras.layers.Dense(16, activation='sigmoid', name='hidden_layer_3'),
    tf.keras.layers.Dense(5, activation='softmax')
])

callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.0001,
    patience=100,
    verbose=0,
    mode="min",
    restore_best_weights=True
)

checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
    loss=tf.keras.losses.CategoricalCrossentropy(),  # Giving 1 to max probability, taking max probability as summation(-p*ln(p))
    metrics = ['accuracy'] # Since using one hot encoding, therefore using binary accuracy
)

history = model.fit(X_train, y_train, epochs=5000, batch_size=len(X_train), verbose=1,
          validation_data = (X_validation, y_validation),
          callbacks=[callback, model_checkpoint_callback]
)

model.load_weights(checkpoint_filepath)

In [ ]:
print(history.history.keys())

In [ ]:
dummy = history.history['accuracy']
avg_training_error=[]
j=1;
for i in dummy:
  avg_training_error.append((j,1-i))
  j+=1
  
plt.plot(*zip(*avg_training_error))
plt.title('Average training error vs epochs')
plt.ylabel('Average training error')
plt.xlabel('epochs')
plt.savefig("/content/average_training_error_vs_epoch.png")

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'])
plt.savefig("/content/model_accuracy.png")
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'])
plt.savefig("/content/model_loss.png")
plt.show()

In [ ]:
model.summary()

In [ ]:
plt.subplot(2,3,1),plt.imshow(X_test[10].reshape(28,28))
print(y_test[10])

In [ ]:
pred = model.predict(X_test)
y_pred_test = []
for dummy in pred:
    dummy1 = (np.argmax(dummy, axis=0))
    y_pred_test.append(dummy1)

y_test_1 = []
for dummy in y_test:
    dummy1 = (np.argmax(dummy, axis=0))
    y_test_1.append(dummy1)

y_pred_test = np.array(y_pred_test)

cf_matrix_test = confusion_matrix(y_test_1, y_pred_test)
print(cf_matrix_test)

x_axis_labels = [2,4,6,7,8]
y_axis_labels = [2,4,6,7,8]
cf_matrix_test_plot = sns.heatmap(cf_matrix_test, xticklabels=x_axis_labels, yticklabels=y_axis_labels, annot=True, fmt='g')   # fmt='g': general format, annot:If True, write the data value in each cell.
fig = cf_matrix_test_plot.get_figure()
fig.savefig("confusion_matrix_test.png") 

In [ ]:
pred = model.predict(X_train)
y_pred = []

for dummy in pred:
    dummy1 = (np.argmax(dummy, axis=0))
    y_pred.append(dummy1)

y_train_1 = []
for dummy in y_train:
    dummy1 = (np.argmax(dummy, axis=0))
    y_train_1.append(dummy1)

y_pred = np.array(y_pred)

cf_matrix_train = confusion_matrix(y_train_1, y_pred)
print(cf_matrix_train)

x_axis_labels = [2,4,6,7,8]
y_axis_labels = [2,4,6,7,8]
cf_matrix_train_plot = sns.heatmap(cf_matrix_train, xticklabels=x_axis_labels, yticklabels=y_axis_labels, annot=True, fmt='g')    # fmt='g': general format, annot:If True, write the data value in each cell.
fig = cf_matrix_train_plot.get_figure()
fig.savefig("confusion_matrix_train.png")

In [ ]:
pred = model.predict(X_validation)
y_pred_validation = []
for dummy in pred:
    dummy1 = (np.argmax(dummy, axis=0))
    y_pred_validation.append(dummy1)

y_validation_1 = []
for dummy in y_validation:
    dummy1 = (np.argmax(dummy, axis=0))
    y_validation_1.append(dummy1)

y_pred_validation = np.array(y_pred_validation)

cf_matrix_validation = confusion_matrix(y_validation_1, y_pred_validation)
print(cf_matrix_validation)

x_axis_labels = [2,4,6,7,8]
y_axis_labels = [2,4,6,7,8]
cf_matrix_validation_plot = sns.heatmap(cf_matrix_validation, xticklabels=x_axis_labels, yticklabels=y_axis_labels, annot=True, fmt='g')   # fmt='g': general format, annot:If True, write the data value in each cell.
fig = cf_matrix_validation_plot.get_figure()
fig.savefig("confusion_matrix_validation.png") 

In [ ]:
loss, acc = model.evaluate(X_test, y_test, verbose=2)
print('Test Accuracy: %.3f' % acc)